Modelo SVM

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

# Cargar los datos desde el archivo CSV
data = pd.read_csv('appliance_usage_data2.csv')

# Dividir los datos en características (X) y etiquetas (y)
X = data[['Hora', 'Probabilidad de fallo', 'Porcentaje de fallos']].values
y = data['Reporte de fallo'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características (normalización)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Crear y entrenar el modelo SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test)
print("Predicción: ", y_pred)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Guardar el modelo en un archivo
joblib.dump(svm_model, 'svm_model.pkl')

Predicción:  [0 1 1 ... 1 0 1]
Accuracy: 0.99984


['svm_model.pkl']